In [1]:
from google.colab import drive
drive.mount('/content/gdrive') #, force_remount=True

Mounted at /content/gdrive


In [2]:
import os
os.chdir('gdrive/My Drive/Colab Notebooks')

In [ ]:
!pip install pymorphy2
!pip install pymystem3==0.1.10
!pip install rusenttokenize
!pip install razdel

In [6]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [179]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter, defaultdict
from pprint import pprint
from nltk import sent_tokenize
punctuation += "«»—…“”"
punct = set(punctuation)
from sklearn.metrics import classification_report, accuracy_score
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from collections import Counter
import math
from nltk.corpus import stopwords

def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

**Задание 1.**

Преобразуйте таблицу с абсолютными частотностями в семинарской тетрадке в таблицу с tfidf значениями. (Таблица - https://i.ibb.co/r5Nc2HC/abs-bow.jpg) Формула tfidf есть в семинаре на картнике с пояснениями на английском. 
Можно считать в google sheets, екселе, питоне или на листочке. Сделайте скрин получившейся таблицы и загрузите картинку на гитхаб, вставьте ссылку в поле ниже.

In [115]:
texts = ['я и ты', 'ты и я', 'я, я и только я', 'только не я', 'он']

In [120]:
def build_terms(corpus):
    terms = {}
    for doc in corpus:
        for word in normalize(doc):
            if word not in terms:
              terms[word] = 1
            else:
              terms[word] += 1
    return terms

terms = build_terms(texts)
terms
vocab = list(terms.keys())
vocab

['я', 'и', 'ты', 'только', 'не', 'он']

In [143]:
def tf(document, terms):
    words = normalize(document)
    total_words = len(words)
    doc_counter = Counter(words)
    for word in doc_counter:
      doc_counter[word] /= total_words
    tfs = [0 for _ in range(len(terms))]
    for i in range(len(terms)):
      tfs[i] = doc_counter[terms[i]]
    return tfs

def count_docs_with_word(word, docs):
    counter = 0
    for doc in docs:
        if word in doc:
            counter += 1
    return counter

def idf(documents, terms):
    idfs = [0 for _ in range(len(terms))]
    total_docs = len(documents)
    for i in range(len(terms)):
      docs_with_word = count_docs_with_word(terms[i], documents)
      idf = 1 + math.log((total_docs+1) / (docs_with_word+1))
      idfs[i] = idf
    return idfs

def td_idf(tf, idf, terms):
    return [tf[i] * idf[i] for i in range(len(terms))]

def build_tfidf(corpus, document, terms):
    doc_tf = tf(document, terms)
    doc_idf = idf(corpus, terms)
    return td_idf(doc_tf, doc_idf, terms)

In [144]:
tf_idf_total = []
for document in texts:
  tf_idf_total.append(build_tfidf(texts, document, vocab))

In [145]:
import pandas as pd
frame = pd.DataFrame(tf_idf_total, columns=terms, index=texts)
frame

,я,и,ты,только,не,он
я и ты,0.394107,0.468488,0.564382,0.000000,0.000000,0.000000
ты и я,0.394107,0.468488,0.564382,0.000000,0.000000,0.000000
"я, я и только я",0.709393,0.281093,0.000000,0.338629,0.000000,0.000000
только не я,0.394107,0.000000,0.000000,0.564382,0.699537,0.000000
он,0.000000,0.000000,0.000000,0.000000,0.000000,2.098612


**Задание 2.**

а) Посчитайте близость между 3 и 12666 текстами в датасете (labeled.csv из семинара) 

б) найдите 3 самых близких текста к тексту номер 43; выведите сами тексты и значения близостей, а не только индексы этих текстов. 

Векторизовать можно любым способом, но постарайтесь, чтобы 1 и 0 не получались в близостях.
Результат запишите в тетрадке.

In [148]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity

In [147]:
df = pd.read_csv('labeled.csv')
df.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [152]:
texts = df.comment
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(texts)

а) Посчитайте близость между 3 и 12666 текстами в датасете (labeled.csv из семинара)

In [153]:
print(texts[3])
print(texts[12666])

Страницу обнови, дебил. Это тоже не оскорбление, а доказанный факт - не-дебил про себя во множественном числе писать не будет. Или мы в тебя верим - это ты и твои воображаемые друзья?

Это не параноик, это дебил



In [162]:
cosine_similarity(X[3], X[12666])

array([[0.27330886]])

б) найдите 3 самых близких текста к тексту номер 43; выведите сами тексты и значения близостей, а не только индексы этих текстов. 

In [164]:
#или argsort()
sims = []
for i in range(len(texts)):
  cossim = cosine_similarity(X[43], X[i])
  sims.append([i, cossim])

In [169]:
sorted(sims, key = lambda x: x[1], reverse=True)[:4]

[[43, array([[1.]])],
 [1986, array([[0.16043982]])],
 [1957, array([[0.11789149]])],
 [6259, array([[0.11409081]])]]

In [173]:
for i in sorted(sims, key = lambda x: x[1], reverse=True)[:4]:
  print(i[0], texts[i[0]])

43 Люди зажрались и охуели если по мнению этих игроков андромеда лучше Антема. Хотя че там с багаутом76 сравнивают вон... Вот оно че оказывается, игроки просто охуели, ну ок. Вот долбанные пидерасы, не хотят покупать такую отличную игру, а еще смеют ругать такой божественный геймплей. Баги, хуевый геймплей поправят, а контент, ну его запилят, через год другой, причем бесплатно! Ага. Стоит лишь потерпеть!

1986 НУ И КАКАЯ МРАЗЬ КИДАЕТ ССЫЛКИ? ОХУЕЛИ ТАМ В КРАЙ УЖЕ?

1957 Че за бригада и че за махоун? Из полицейской академии?
6259 Герка ебет только даунов которые игрли а него. Ибо и геймплей и сюжетто кусок говна.



**Задание 3.**

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из 2ch_corpus.txt (второй семинар) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?
*Можете предсказывать не все данные, но не менее 2500.

Требования к классификаторам:

а) один должен использовать CountVectorizer, другой TfidfVectorizer

б) у векторазера должны быть вручную заданы как минимум 5 параметров

в) у классификатора должно быть задано вручную как минимум 2 параметра

г)  f1 мера каждого из классификаторов должна быть минимум 0.75

In [295]:
vect_1 = CountVectorizer(min_df=2, max_df=0.9, stop_words=stopwords.words("russian"), binary='bool', lowercase='bool')

X_train, X_test, y_train, y_test = train_test_split(df.comment, df.toxic, test_size=0.1, random_state=42)
X_train = vect_1.fit_transform(X_train)
X_test = vect_1.transform(X_test)

In [296]:
clf = LogisticRegression(C=0.5, class_weight='balanced', solver='saga', max_iter=500)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

         0.0       0.90      0.87      0.88       971
         1.0       0.75      0.79      0.77       471

    accuracy                           0.84      1442
   macro avg       0.82      0.83      0.83      1442
weighted avg       0.85      0.84      0.85      1442



In [293]:
vect_2 = TfidfVectorizer(min_df=2, max_df=0.9, stop_words=stopwords.words("russian"), binary='bool', lowercase='bool')

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(df.comment, df.toxic, test_size=0.1, random_state=42)
X_train_2 = vect_2.fit_transform(X_train_2)
X_test_2 = vect_2.transform(X_test_2)

In [254]:
clf_2 = MultinomialNB(alpha=0.5, fit_prior='bool', class_prior=[1,1])
clf_2.fit(X_train_2, y_train_2)
preds_2 = clf_2.predict(X_test_2)
print(classification_report(y_test_2, preds_2))

              precision    recall  f1-score   support

         0.0       0.88      0.93      0.90       971
         1.0       0.83      0.75      0.79       471

    accuracy                           0.87      1442
   macro avg       0.86      0.84      0.85      1442
weighted avg       0.87      0.87      0.87      1442



In [259]:
dvach = open('2ch_corpus.txt').readlines()[:3000]

In [290]:
def get_probs(texts, vect, model):
  X_test = vect.transform(texts)
  probs = model.predict_proba(X_test)
  top10 = probs[:,1].argsort()[::-1][:10]
  top10_probs = probs[top10, 1]

  return top10, top10_probs

In [291]:
#топ для первой модели
top10_1, probs_1 = get_probs(dvach, vect_1, clf)

In [275]:
for i in range(10):
  print(probs_1[i])
  print(top10_1[i], dvach[top10_1[i]], '\n')

0.9999638953752288
256  >Scala, Clojure, Erlang, Elixir, Common Lisp, Haskell, Ocaml, F#, Elm, Swift, Go, Rust, D, Nim, Scheme, SmalltalkМолодец, перечислил сложные для старта языки + на которых вакансий хуй хуй да нихуя. Сам-то с какого языка начинал? И наверное в гарварде и mit дауны сидят и учат людей сям и питону, сука на дваче, что не доска, так куча нитаких, как все даунов на хайпе, которые толкают свое говно. Нахуй иди.
 

0.9999514029087191
2810  - Что можете сказать о стиле Тарантиныча?- Кого блять?- Квентина Тарантино.- Так и говори, блять.- Почему его героев, говорящих обо всём и ни о чём, так приятно слушать?- Потому что диалоги остроумные. Из двух слов. Умные. И острые. - Как лист салата?- Что "как лист салата"?- Умные.- Что за хуйню ты сейчас несешь?- Мы же сейчас о тарантиновских диалогах с тобой разговариваем?- Да, блять, о них мы и разговариваем. Ты кокаином обнюханный что ли я не пойму?- При чем здесь кокаин? Его диалоги без штампов, без стереотипов, и точно бьющие в 

In [270]:
#топ для второй модели
top10_2, probs_2 = get_probs(dvach, vect_2, clf_2)

In [274]:
for i in range(10):
  print(probs_2[i])
  print(top10_2[i], dvach[top10_2[i]], '\n')

0.9955543303551457
456   мамка твоя мне заработала своей пиздой вонючей тварь вонючая, мразь поганая, гандон, говно, пидрила, ану иди сюда
 

0.9946419106455782
2467  Иди нахуй дурачек ебаный пидорас ну чтоьты за скотина ебаная оп выше написал что нашел ее а теперь иди нахкй
 

0.9925055052406684
1372  Боже, ебаные нытики. Да не сдохнешь ты, мразь. Не ной.
 

0.9858169539344932
523  Где-то в этой пасте шлюха пиздит.
 

0.9853922197713505
2062  Фу блядь нахуй всратый.
 

0.9850055463751813
1141  Ишак ебаный, какая в пизду смекта, иди бабку свою лечи, и смотри как лечися острый панкриотит (хирург-кун)
 

0.9849170754518075
1279  Вызывай, подохнешь же, сука.
 

0.9845573582923393
1607  Кукарекает твой дружок-пидор, ну и ты вместе с ним. Слизываешь поди малафью с его очка сидишь, распизделся тута.
 

0.9839760537467467
725  Нахуй съеби, хуесос /po/рашный.
 

0.9839006553295898
979  Сука, этот тред - просто кладезь лулзов!
 



В получившихся топах есть общие тексты: первые два текста во 2-ом топе встречаются также и в 1-ом топе.

In [278]:
c = set(top10_1).intersection(set(top10_2))
print(c)

{456, 2467}


Все тексты в обоих топах токсичные. Можно отметить, что в первом топе тексты длинее, чем во втором, и, есть ощущение, что во втором топе тексты более токсичные (особенно если учесть, что в текстах из первого топа присутствуют какие-то дополнительные мысли и рассуждения, а во втором топе много просто ругани)